In [2]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics.pairwise import rbf_kernel, polynomial_kernel, linear_kernel

In [3]:
class WSVM:
    def __init__(self, gamma='scale', degree=3, coef0=0.0):
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
        self.alpha = None
        self.b = 0
        self.X = None
        self.y = None
        self.weights = None

    def _get_gamma(self, X):
        if isinstance(X, np.ndarray):
            n_features = X.shape[1]
        else:
            raise ValueError("X must be a numpy.ndarray")
        
        if self.gamma == 'scale':
            return 1.0 / (n_features * np.var(X, axis=0).mean())
        elif self.gamma == 'auto':
            return 1.0 / n_features
        elif isinstance(self.gamma, (int, float)):
            return self.gamma
        else:
            raise ValueError("gamma must be 'scale', 'auto', or a float")
        
        
    def fit(self, Y, X, W, weights):
        self.Y = Y
        self.X = X
        self.W = W
        self.weights = weights
        n_samples = X.shape[0]
        
        # Compute the kernel matrix
        if self.kernel == 'rbf':
            gamma = self._get_gamma(X)
            K = rbf_kernel(X, gamma=gamma)
        elif self.kernel == 'poly':
            K = polynomial_kernel(X, degree=self.degree, coef0=self.coef0)
        elif self.kernel == 'linear':
            K = linear_kernel(X)
        else:
            raise ValueError("Unsupported kernel type")
        
        # Solve the quadratic programming problem using sklearn's SVC
        # We use a workaround to handle instance weights
        self.model = SVC(kernel=self.kernel, C=self.C)
        self.model.fit(X, W, sample_weight=weights)
        
        # Extract dual coefficients (alphas) and intercept (b)
        self.alpha = np.abs(self.model.dual_coef_[0])
        self.ate = self.estimate_ate(Y, W, X, self.alpha)
        self.diff = self.estimate_diff(Y, W, X, self.alpha)
        self.effective_sample_set = self.estimate_ess(Y, W, X, self.alpha)
        
        return self
    
    
    def estimate_ate(self, Y, W, X, alpha):

        support_indices = self.model.support_
    
        # 计算权重
        weights = np.zeros(X.shape[0])
        weights[support_indices] = np.abs(alpha)
        alpha = weights / np.sum(weights)
        
        treated_mean = np.sum(Y[W == 1] * alpha[W == 1]) / np.sum(alpha[W == 1])
        control_mean = np.sum(Y[W == -1] * alpha[W == -1]) / np.sum(alpha[W == -1])
        return treated_mean - control_mean
    

    def estimate_diff(self, Y, W, X, alpha):

        support_indices = self.model.support_
    
        # 计算权重
        weights = np.zeros(X.shape[0])
        weights[support_indices] = np.abs(alpha)
        alpha = weights / np.sum(weights)
        
        treated_mean = np.mean(Y[W == 1])
        control_mean = np.mean(Y[W == -1])
        
        treated_std = np.std(Y[W == 1], ddof=1)
        control_std = np.std(Y[W == -1], ddof=1)

        normed_diff = np.abs(treated_mean - control_mean) / np.sqrt((treated_std**2 + control_std**2) / 2)

        return normed_diff
    
    def estimate_ess(self, Y, W, X, alpha):

        support_indices = self.model.support_
    
        # 计算权重
        weights = np.zeros(X.shape[0])
        weights[support_indices] = np.abs(alpha)
        alpha = weights / np.sum(weights)
        
        treated_ess = (np.sum(alpha[W == 1]))**2 / np.sum(alpha[W == 1]**2)
        control_ess = (np.sum(alpha[W == -1]))**2 / np.sum(alpha[W == -1]**2)

        return treated_ess + control_ess
    

    def predict(self, X):
        if self.kernel == 'rbf':
            gamma = self._get_gamma(self.X)
            K = rbf_kernel(X, self.X, gamma=gamma)
        elif self.kernel == 'poly':
            K = polynomial_kernel(X, self.X, degree=self.degree, coef0=self.coef0)
        elif self.kernel == 'linear':
            K = linear_kernel(X, self.X)
        else:
            raise ValueError("Unsupported kernel type")

    def path(self, simulate_data_A, C, kernel, w1_values, w_minus1_values):
        X = simulate_data_A.drop(columns=['T', 'Y'])
        T = simulate_data_A['T']
        Y = simulate_data_A['Y']
        W = 2 * T - 1

        self.C = C
        self.kernel = kernel

        if not isinstance(X, np.ndarray):
            X = np.array(X)


        path = []
        
        for w1 in w1_values:
            for w_minus1 in w_minus1_values:
                weights = np.where(W == 1, w1, w_minus1).astype(float)
                self.fit(Y, X, W, weights)
                path.append((w1, w_minus1, self.ate, self.diff, self.effective_sample_set))
        
        return path

In [4]:
num_datasets = 10
simulation_results_A = {n: [] for n in range(num_datasets)}

# Track the solution path
kernels = ['rbf']
C_range = np.logspace(-3, 3, 10)
w1_values = np.linspace(0.1, 2.0, 10)
w_minus1_values = np.linspace(0.1, 2.0, 10)

results = {kernel: [] for kernel in kernels}

for n in range(num_datasets):
    simulate_data_A = pd.read_csv('../data_A/data_scenario_G_n_500_dataset_{}.csv'.format(n))

    for kernel in kernels:
        # Initialize WSVM
        wsvm = WSVM(gamma='scale')
        for C in C_range:
            path = wsvm.path(simulate_data_A, C, kernel, w1_values, w_minus1_values)

            # Print the path
            for step, (w1, w_minus1, ate, diff, ess) in enumerate(path):
                results[kernel].append((C, w1, w_minus1, ate, diff, ess))
                print(f"Step {step}: C = {C}, kernel = {kernel}, w1={w1}, w_minus1={w_minus1}, ATE = {ate}, DIM = {diff}, ESS = {ess}")
    
    simulation_results_A[n] = results 


Step 0: C = 0.001, kernel = rbf w1=0.1, w_minus1=0.1, ATE = -0.40825174467420505, DIM = 0.9648479094942628,ESS = 409.9999999999998
Step 1: C = 0.001, kernel = rbf w1=0.1, w_minus1=0.3111111111111111, ATE = -0.4025420875122522, DIM = 0.9648479094942628,ESS = 430.59728767913816
Step 2: C = 0.001, kernel = rbf w1=0.1, w_minus1=0.5222222222222223, ATE = -0.4027687614165532, DIM = 0.9648479094942628,ESS = 414.9854465607925
Step 3: C = 0.001, kernel = rbf w1=0.1, w_minus1=0.7333333333333333, ATE = -0.4021106815940364, DIM = 0.9648479094942628,ESS = 412.12651413189764
Step 4: C = 0.001, kernel = rbf w1=0.1, w_minus1=0.9444444444444444, ATE = -0.4021106815940364, DIM = 0.9648479094942628,ESS = 412.12651413189764
Step 5: C = 0.001, kernel = rbf w1=0.1, w_minus1=1.1555555555555557, ATE = -0.4021106815940364, DIM = 0.9648479094942628,ESS = 412.12651413189764
Step 6: C = 0.001, kernel = rbf w1=0.1, w_minus1=1.3666666666666667, ATE = -0.4021106815940364, DIM = 0.9648479094942628,ESS = 412.126514131

In [ ]:
##PLOT ATE
def plot_ate(simulation_results):

    kernels = ['linear', 'poly', 'rbf']

    # fig, axes = plt.subplots(1, len(kernels), figsize=(4 * len(kernels), 4), sharey=True)
    # axes = axes.flatten()

    # 创建一个子图
    fig, axes = plt.subplots(1, 3,figsize=(12, 4))
    fig.subplots_adjust(hspace=0.4, wspace=0.3)

    i= 0

    # 过滤出当前kernel的所有模型
    for kernel in kernels:
        i = i
        C_values = []
        kernel_ate_estimates = []
        for results in simulation_results.values():
            if kernel in results:
                for model in results[kernel]:
                    C_values.append(model[0])
                    kernel_ate_estimates.append(model[3])

        data = pd.DataFrame({'C': C_values, 'ATE': kernel_ate_estimates})
        grouped_ate = data.groupby('C')['ATE'].apply(list).tolist()
        print(grouped_ate)

        
        # 绘制箱型图
        axes[i].boxplot(grouped_ate)
        axes[i].axhline(y=-0.4, color='r', linestyle='--', label='True ATE (A)')
        axes[i].set_xscale('log')
        axes[i].set_title(f'{kernel}')
        axes[i].set_xlabel(r'$\lambda^{-1}$')
        axes[i].set_ylabel('ATE')
        axes[i].legend()

        i += 1

    plt.tight_layout()
    plt.show()

In [ ]:
plot_ate(simulation_results_A)

In [5]:
import json
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    """自定义JSON编码器，用于处理numpy数据类型"""
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NumpyEncoder, self).default(obj)

# 假设simulation_results_A是一个包含numpy数据类型的列表或字典
data = simulation_results_A

# 使用自定义编码器将数据转换为JSON格式的字符串
data_str = json.dumps(data, cls=NumpyEncoder, indent=4)

# 保存到TXT文件
with open('simulation_results_A.json', 'w') as f:
    f.write(data_str)